In [1]:
ls


drive/  sample_data/


In [2]:
cd drive/MyDrive/ISIC_Data_Segmentation/


/content/drive/MyDrive/ISIC_Data_Segmentation


In [3]:
ls

data/  logs/  trained_models/


In [2]:
#Import all dependencies

import os
import PIL

In [5]:
os.listdir('data')

['train', 'test', 'val']

In [3]:
print("Training Images:\t",(len(os.listdir(r"data/train/images"))),"\t","Groundtruth Masks:\t ",((len(os.listdir(r"data/train/masks")))))
print("Validation Images:\t",(len(os.listdir(r"data/val/images"))),"\t","Groundtruth Masks:\t ",((len(os.listdir(r"data/val/masks")))))
print("Testing Images: \t",(len(os.listdir(r"data/test/images"))),"\t","Groundtruth Masks:\t ",((len(os.listdir(r"data/test/masks")))))


Training Images:	 323 	 Groundtruth Masks:	  323
Validation Images:	 46 	 Groundtruth Masks:	  46
Testing Images: 	 89 	 Groundtruth Masks:	  89


In [7]:
#check image mode
img = os.listdir('data/train/images')[0]
with PIL.Image.open('data/train/images/'+img) as img : 
  print("train image: ",img.mode)

img = os.listdir('data/train/masks')[0]
with PIL.Image.open('data/train/masks/'+img) as img :
  print("training ground truth image: ",img.mode)

img = os.listdir('data/test/images')[0]
with PIL.Image.open('data/test/images/'+img) as img :
  print("test image: ",img.mode)


img = os.listdir('data/val/images')[0]
with PIL.Image.open('data/val/images/'+img) as img :
  print("validation image: ",img.mode)

img = os.listdir('data/val/masks')[0]
with PIL.Image.open('data/val/masks/'+img) as img :
  print("validation ground truth image: ",img.mode)

train image:  RGB
training ground truth image:  RGB
test image:  RGB
validation image:  RGB
validation ground truth image:  RGB


In [25]:
#Convert from grayscale to RGB (R=G=B=gray)
temp = ['train','val']
for repo in temp :
  for image in os.listdir("data/"+repo+"/masks"):
      with PIL.Image.open("data/"+repo+"/masks/"+image) as img :
        rgbimg = PIL.Image.new("RGB", img.size)
        rgbimg.paste(img)
        rgbimg.save("data/"+repo+"/masks/"+image)



In [8]:
#validating everything is in RGB & not in RGBA or L

img = os.listdir('data/train/images')[0]
img = PIL.Image.open('data/train/images/'+img)
print("train image: ",img.mode)

img = os.listdir('data/train/masks')[0]
img = PIL.Image.open('data/train/masks/'+img)
print("training ground truth image: ",img.mode)

img = os.listdir('data/test/images')[0]
img = PIL.Image.open('data/test/images/'+img)
print("test image: ",img.mode)

img = os.listdir('data/val/images')[0]
img = PIL.Image.open('data/val/images/'+img)
print("validation image: ",img.mode)

img = os.listdir('data/val/masks')[0]
img = PIL.Image.open('data/val/masks/'+img)
print("validation ground truth image: ",img.mode)

train image:  RGB
training ground truth image:  RGB
test image:  RGB
validation image:  RGB
validation ground truth image:  RGB


In [4]:
#Load Dataset

import os
import torch
from torch.utils.data import Dataset
from torchvision import transforms
from PIL import Image
import numpy as np
#
class myDataset(Dataset):
    def __init__(self, root ,train=True, transform = None):
        Dataset.__init__(self)
        images_dir = os.path.join(root,'images')
        images = os.listdir(images_dir)
        self.images = [os.path.join(images_dir, k) for k in images]
        self.images.sort()
        if train:
            masks_dir = os.path.join(root,'masks')
            masks = os.listdir(masks_dir)
            self.masks = [os.path.join(masks_dir, k) for k in masks]
            self.masks.sort()

        self.transforms = transform
        self.train = train
        
    def __getitem__(self, index):
        image_path = self.images[index]
        
        image = Image.open(image_path).resize([512,512])
        if self.transforms is not None:
            image = self.transforms(image)
        image = image
        if self.train :
            mask_path = self.masks[index]
            mask = Image.open(mask_path).resize([512,512])
            if self.transforms is not None:
                mask = self.transforms(mask)
                mask = mask.mean(dim=0).view(1,512,512)
                mask[mask>0] = 0
                mask[mask<0] = 1
                
            return image, mask
        return image
    
    def __len__(self):
        return len(self.images)
    

In [5]:
#Loss Functions

import torch

def dice_loss(pred, target):
    """This definition generalize to real valued pred and target vector.
This should be differentiable.
    pred: tensor with first dimension as batch
    target: tensor with first dimension as batch
    """
    #target[target==0] = -1

    smooth = 1.

    # have to use contiguous since they may from a torch.view op
    iflat = pred.contiguous().view(-1)
    tflat = target.contiguous().view(-1)
    intersection = (iflat * tflat).sum()

    return 1 - ((2. * intersection + smooth) / (iflat.sum() + tflat.sum() + smooth))

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable

class FocalLoss(nn.Module):
    def __init__(self, gamma=0, alpha=None, size_average=True):
        super(FocalLoss, self).__init__()
        self.gamma = gamma
        self.alpha = alpha
        if isinstance(alpha,(float,int)): self.alpha = torch.Tensor([alpha,1-alpha])
        if isinstance(alpha,list): self.alpha = torch.Tensor(alpha)
        self.size_average = size_average

    def forward(self, input, target):
        if input.dim()>2:
            input = input.view(input.size(0),input.size(1),-1)  # N,C,H,W => N,C,H*W
            input = input.transpose(1,2)    # N,C,H*W => N,H*W,C
            input = input.contiguous().view(-1,input.size(2))   # N,H*W,C => N*H*W,C
        target = target.view(-1,1)

        logpt = F.log_softmax(input)
        logpt = logpt.gather(1,target)
        logpt = logpt.view(-1)
        pt = Variable(logpt.data.exp())

        if self.alpha is not None:
            if self.alpha.type()!=input.data.type():
                self.alpha = self.alpha.type_as(input.data)
            at = self.alpha.gather(0,target.data.view(-1))
            logpt = logpt * Variable(at)

        loss = -1 * (1-pt)**self.gamma * logpt
        if self.size_average: return loss.mean()
        else: return loss.sum()
        
from __future__ import print_function, division

import torch
from torch.autograd import Variable
import torch.nn.functional as F
import numpy as np
try:
    from itertools import  ifilterfalse
except ImportError: # py3k
    from itertools import  filterfalse as ifilterfalse


def lovasz_grad(gt_sorted):
    """
    Computes gradient of the Lovasz extension w.r.t sorted errors
    See Alg. 1 in paper
    """
    p = len(gt_sorted)
    gts = gt_sorted.sum()
    intersection = gts - gt_sorted.float().cumsum(0)
    union = gts + (1 - gt_sorted).float().cumsum(0)
    jaccard = 1. - intersection / union
    if p > 1: # cover 1-pixel case
        jaccard[1:p] = jaccard[1:p] - jaccard[0:-1]
    return jaccard


def iou_binary(preds, labels, EMPTY=1., ignore=None, per_image=True):
    """
    IoU for foreground class
    binary: 1 foreground, 0 background
    """
    if not per_image:
        preds, labels = (preds,), (labels,)
    ious = []
    for pred, label in zip(preds, labels):
        intersection = ((label == 1) & (pred == 1)).sum()
        union = ((label == 1) | ((pred == 1) & (label != ignore))).sum()
        if not union:
            iou = EMPTY
        else:
            iou = float(intersection) / union
        ious.append(iou)
    iou = mean(ious)    # mean accross images if per_image
    return 100 * iou


def iou(preds, labels, C, EMPTY=1., ignore=None, per_image=False):
    """
    Array of IoU for each (non ignored) class
    """
    if not per_image:
        preds, labels = (preds,), (labels,)
    ious = []
    for pred, label in zip(preds, labels):
        iou = []    
        for i in range(C):
            if i != ignore: # The ignored label is sometimes among predicted classes (ENet - CityScapes)
                intersection = ((label == i) & (pred == i)).sum()
                union = ((label == i) | ((pred == i) & (label != ignore))).sum()
                if not union:
                    iou.append(EMPTY)
                else:
                    iou.append(float(intersection) / union)
        ious.append(iou)
    ious = map(mean, zip(*ious)) # mean accross images if per_image
    return 100 * np.array(ious)


# --------------------------- BINARY LOSSES ---------------------------


def lovasz_hinge(logits, labels, per_image=True, ignore=None):
    """
    Binary Lovasz hinge loss
      logits: [B, H, W] Variable, logits at each pixel (between -\infty and +\infty)
      labels: [B, H, W] Tensor, binary ground truth masks (0 or 1)
      per_image: compute the loss per image instead of per batch
      ignore: void class id
    """

    
    logits = 1. - logits
    labels = 1. - labels
    #------------------#

    if per_image:
        loss = mean(lovasz_hinge_flat(*flatten_binary_scores(log.unsqueeze(0), lab.unsqueeze(0), ignore))
                          for log, lab in zip(logits, labels))
    else:
        loss = lovasz_hinge_flat(*flatten_binary_scores(logits, labels, ignore))
    return loss


def lovasz_hinge_flat(logits, labels):
    """
    Binary Lovasz hinge loss
      logits: [P] Variable, logits at each prediction (between -\infty and +\infty)
      labels: [P] Tensor, binary ground truth labels (0 or 1)
      ignore: label to ignore
    """
    if len(labels) == 0:
        # only void pixels, the gradients should be 0
        return logits.sum() * 0.
    signs = 2. * labels.float() - 1.
    errors = (1. - logits * Variable(signs))
    errors_sorted, perm = torch.sort(errors, dim=0, descending=True)
    perm = perm.data
    gt_sorted = labels[perm]
    grad = lovasz_grad(gt_sorted)
    loss = torch.dot(F.relu(errors_sorted), Variable(grad))
    return loss


def flatten_binary_scores(scores, labels, ignore=None):
    """
    Flattens predictions in the batch (binary case)
    Remove labels equal to 'ignore'
    """
    scores = scores.view(-1)
    labels = labels.view(-1)
    if ignore is None:
        return scores, labels
    valid = (labels != ignore)
    vscores = scores[valid]
    vlabels = labels[valid]
    return vscores, vlabels


class StableBCELoss(torch.nn.modules.Module):
    def __init__(self):
         super(StableBCELoss, self).__init__()
    def forward(self, input, target):
         neg_abs = - input.abs()
         loss = input.clamp(min=0) - input * target + (1 + neg_abs.exp()).log()
         return loss.mean()


def binary_xloss(logits, labels, ignore=None):
    """
    Binary Cross entropy loss
      logits: [B, H, W] Variable, logits at each pixel (between -\infty and +\infty)
      labels: [B, H, W] Tensor, binary ground truth masks (0 or 1)
      ignore: void class id
    """
    logits, labels = flatten_binary_scores(logits, labels, ignore)
    loss = StableBCELoss()(logits, Variable(labels.float()))
    return loss


# --------------------------- MULTICLASS LOSSES ---------------------------


def lovasz_softmax(probas, labels, only_present=False, per_image=False, ignore=None):
    """
    Multi-class Lovasz-Softmax loss
      probas: [B, C, H, W] Variable, class probabilities at each prediction (between 0 and 1)
      labels: [B, H, W] Tensor, ground truth labels (between 0 and C - 1)
      only_present: average only on classes present in ground truth
      per_image: compute the loss per image instead of per batch
      ignore: void class labels
    """
    if per_image:
        loss = mean(lovasz_softmax_flat(*flatten_probas(prob.unsqueeze(0), lab.unsqueeze(0), ignore), only_present=only_present)
                          for prob, lab in zip(probas, labels))
    else:
        loss = lovasz_softmax_flat(*flatten_probas(probas, labels, ignore), only_present=only_present)
    return loss


def lovasz_softmax_flat(probas, labels, only_present=False):
    """
    Multi-class Lovasz-Softmax loss
      probas: [P, C] Variable, class probabilities at each prediction (between 0 and 1)
      labels: [P] Tensor, ground truth labels (between 0 and C - 1)
      only_present: average only on classes present in ground truth
    """
    if probas.numel() == 0:
        # only void pixels, the gradients should be 0
        return probas * 0.
    C = probas.size(1)
    
    C = probas.size(1)
    losses = []
    for c in range(C):
        fg = (labels == c).float() # foreground for class c
        if only_present and fg.sum() == 0:
            continue
        errors = (Variable(fg) - probas[:, c]).abs()
        errors_sorted, perm = torch.sort(errors, 0, descending=True)
        perm = perm.data
        fg_sorted = fg[perm]
        losses.append(torch.dot(errors_sorted, Variable(lovasz_grad(fg_sorted))))
    return mean(losses)


def flatten_probas(probas, labels, ignore=None):
    """
    Flattens predictions in the batch
    """
    B, C, H, W = probas.size()
    probas = probas.permute(0, 2, 3, 1).contiguous().view(-1, C)  # B * H * W, C = P, C
    labels = labels.view(-1)
    if ignore is None:
        return probas, labels
    valid = (labels != ignore)
    vprobas = probas[valid.nonzero().squeeze()]
    vlabels = labels[valid]
    return vprobas, vlabels

def xloss(logits, labels, ignore=None):
    """
    Cross entropy loss
    """
    return F.cross_entropy(logits, Variable(labels), ignore_index=255)


# --------------------------- HELPER FUNCTIONS ---------------------------
def isnan(x):
    return x != x
    
    
def mean(l, ignore_nan=True, empty=0):
    """
    nanmean compatible with generators.
    """
    l = iter(l)
    if ignore_nan:
        l = ifilterfalse(isnan, l)
    try:
        n = 1
        acc = next(l)
    except StopIteration:
        if empty == 'raise':
            raise ValueError('Empty mean')
        return empty
    for n, v in enumerate(l, 2):
        acc += v
    if n == 1:
        return acc
    return acc / n




In [6]:
#Unet Model

import torch.nn as nn
import torch
from torch import autograd


class ConvolutionBlock(nn.Module):
    def __init__(self, in_ch, out_ch):
        super(ConvolutionBlock, self).__init__()
        self.conv = nn.Sequential(
            nn.Conv2d(in_ch, out_ch, 3, padding=1),
            nn.BatchNorm2d(out_ch),
            nn.ReLU(inplace=True),
            nn.Conv2d(out_ch, out_ch, 3, padding=1),
            nn.BatchNorm2d(out_ch),
            nn.ReLU(inplace=True)
        )

    def forward(self, input):
        return self.conv(input)

class Unet(nn.Module):
    def __init__(self, in_ch, out_ch):
        super(Unet, self).__init__()
        self.Convolution1 = ConvolutionBlock(in_ch, 64)  
        self.maxpooling1 = nn.MaxPool2d(2)
        self.Convolution2 = ConvolutionBlock(64, 128)
        self.maxpooling2 = nn.MaxPool2d(2)
        self.Convolution3 = ConvolutionBlock(128, 256)
        self.maxpooling3 = nn.MaxPool2d(2)
        self.Convolution4 = ConvolutionBlock(256, 512)
        self.maxpooling4 = nn.MaxPool2d(2)
        self.Convolution5 = ConvolutionBlock(512, 1024)

        self.maxpooling6 = nn.ConvTranspose2d(1024, 512, 2, stride=2)
        self.Convolution6 = ConvolutionBlock(1024, 512)
        self.maxpooling7 = nn.ConvTranspose2d(512, 256, 2, stride=2)
        self.Convolution7 = ConvolutionBlock(512, 256)
        self.maxpooling8 = nn.ConvTranspose2d(256, 128, 2, stride=2)
        self.Convolution8 = ConvolutionBlock(256, 128)
        self.maxpooling9 = nn.ConvTranspose2d(128, 64, 2, stride=2)
        self.Convolution9 = ConvolutionBlock(128, 64)
        self.Convolution10 = nn.Conv2d(64, out_ch, 1)
        
    def forward(self, x):
        conv1 = self.Convolution1(x)
        pool1 = self.maxpooling1(conv1)
        conv2 = self.Convolution2(pool1)
        pool2 = self.maxpooling2(conv2)
        conv3 = self.Convolution3(pool2)
        pool3 = self.maxpooling3(conv3)
        conv4 = self.Convolution4(pool3)
        pool4 = self.maxpooling4(conv4)
        conv5 = self.Convolution5(pool4)
        up_6 = self.maxpooling6(conv5)
        merge6 = torch.cat([up_6, conv4], dim=1)
        conv6 = self.Convolution6(merge6)
        up_7 = self.maxpooling7(conv6)
        merge7 = torch.cat([up_7, conv3], dim=1)
        conv7 = self.Convolution7(merge7)
        up_8 = self.maxpooling8(conv7)
        merge8 = torch.cat([up_8, conv2], dim=1)
        conv8 = self.Convolution8(merge8)
        up_9 = self.maxpooling9(conv8)
        merge9 = torch.cat([up_9, conv1], dim=1)
        conv9 = self.Convolution9(merge9)
        conv10 = self.Convolution10(conv9)
        out = nn.Softmax2d()(conv10)
        return out

In [7]:
unet = Unet(3,2).cuda()

In [8]:
from torchsummary import summary
summary(unet,(3,512,512))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 64, 512, 512]           1,792
       BatchNorm2d-2         [-1, 64, 512, 512]             128
              ReLU-3         [-1, 64, 512, 512]               0
            Conv2d-4         [-1, 64, 512, 512]          36,928
       BatchNorm2d-5         [-1, 64, 512, 512]             128
              ReLU-6         [-1, 64, 512, 512]               0
  ConvolutionBlock-7         [-1, 64, 512, 512]               0
         MaxPool2d-8         [-1, 64, 256, 256]               0
            Conv2d-9        [-1, 128, 256, 256]          73,856
      BatchNorm2d-10        [-1, 128, 256, 256]             256
             ReLU-11        [-1, 128, 256, 256]               0
           Conv2d-12        [-1, 128, 256, 256]         147,584
      BatchNorm2d-13        [-1, 128, 256, 256]             256
             ReLU-14        [-1, 128, 2

In [9]:
#Utility functions
import torch
import torch.nn as nn
from torchvision import transforms
from torch.utils.data import Dataset


transform=transforms.Compose([
    transforms.ToTensor(), 
    transforms.Normalize(mean=[.5,.5,0.5],std=[.5,.5,0.5]) 
])

def cal_iou(model, dataset):
    pa=mpa=miou=fwiou=0.
    for img, mask in dataset:
        mask = mask.cuda()
        with torch.no_grad():
            pred = model(img.unsqueeze(0).cuda())
            pred = torch.argmax(pred, 1).float()
        pa += get_pa(pred, mask)
        mpa += get_mpa(pred, mask)
        miou += get_miou(pred, mask)
        fwiou += get_fwiou(pred, mask)
    lenth = len(dataset)
    pa /= lenth
    mpa /= lenth
    miou /= lenth
    fwiou /= lenth
    return pa.item(), mpa.item(), miou.item(), fwiou.item()

def get_pa(pred, mask):
    return (pred==mask).sum().float()/(512*512)


def get_mpa(pred, mask):
    pred_crack = pred
    pred_fine = 1-pred
    mask_crack = mask
    mask_fine = 1-mask
    return (pred_crack*mask_crack).sum().float()/\
            (mask_crack.sum())/2 +\
            (pred_fine*mask_fine).sum().float()/\
            (mask_fine.sum())/2


def get_miou(pred, mask):

    pred_crack = pred
    pred_fine = 1-pred
    mask_crack = mask
    mask_fine = 1-mask
    return (pred_crack*mask_crack).sum().float()/\
            ((mask_crack+pred_crack)!=0).sum()/2+\
            (pred_fine*mask_fine).sum().float()/\
            ((mask_fine+pred_fine)!=0).sum()/2


def get_fwiou(pred, mask):
    pred_crack = pred
    pred_fine = 1-pred
    mask_crack = mask
    mask_fine = 1-mask
    return  mask_crack.sum()*(pred_crack*mask_crack).sum().float()/\
            ((mask_crack+pred_crack)!=0).sum()/(512*512)+\
            mask_fine.sum()*(pred_fine*mask_fine).sum().float()/\
            ((mask_fine+pred_fine)!=0).sum()/(512*512)


def onehot(masks):
    masks_t = torch.zeros(masks.size(0), 2, 
                    masks.size(2), masks.size(3)).cuda()
    masks_t[:,0,:,:][masks[:,0,:,:]==0] = 1
    masks_t[:,1,:,:][masks[:,0,:,:]==1] = 1   
    return masks_t

In [10]:
#training

import logging
import copy
import time

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from PIL import Image
import numpy as np





batch_size = 4
num_epochs = [30]
num_workers = 2
lr = 0.0001

losslist = [ 'bce'] #can iterate through multiple losses, for final report
optimlist = ['adam'] #can iterate through multiple optimisers, for final report
iflog = True

train_dataset = myDataset('data/train', transform=transform)
val_dataset = myDataset('data/val', transform=transform)
train_loader = DataLoader(dataset=train_dataset,
                            batch_size=batch_size,
                            shuffle=True,
                            num_workers=num_workers)
val_loader = DataLoader(dataset=val_dataset,
                            batch_size=1,
                            shuffle=False)
criterion = nn.BCELoss()
focallos = FocalLoss(gamma=2)

epoidx = -1
for los in losslist:
    for opt in optimlist:
        start =  time.time()
        print("Using loss function: ",los," & Optimiser: ", opt)
        torch.manual_seed(77)
        torch.cuda.manual_seed(77)
        unet = Unet(3,2).cuda()
        history = []
        if 'adam' in opt :
            optimizer = torch.optim.Adam(unet.parameters(), lr=lr)
        elif 'sgd' in opt:
            optimizer = torch.optim.SGD(unet.parameters(), lr=10*lr, momentum=0.9)

        logging.basicConfig(filename='./logs/logger_unet.log', level=logging.INFO)

        total_step = len(train_loader)
        epoidx += 1
        for epoch in range(num_epochs[epoidx]):
            totalloss = 0
            for i, (images, masks) in enumerate(train_loader):
                images = images.cuda()
                masks = masks.cuda()
                outputs = unet(images)
                if 'bce' in los :
                    masks = onehot(masks)              
                    loss = criterion(outputs, masks)
                elif 'dice' in los :
                    masks = onehot(masks)              
                    loss = dice_loss(outputs, masks)
                elif 'lovasz' in los :
                    masks = onehot(masks)          
                    loss = L.lovasz_hinge(outputs, masks)
                elif 'focal' in los :
                    loss = focallos(outputs, masks.long())

                totalloss += loss*images.size(0)

                optimizer.zero_grad()
                loss.backward()
                optimizer.step()
                #print("epoch : ", epoch, "Loss: ",totalloss)
                if i+1 == total_step:
                    train_pa, train_mpa, train_miou, train_fwiou = \
                                        cal_iou(unet,train_dataset)
                    val_pa, val_mpa, val_miou, val_fwiou = \
                                        cal_iou(unet,val_dataset)
                    history.append([totalloss.item()/len(train_dataset), 
                                    train_pa, train_mpa, train_miou, train_fwiou,
                                    val_pa, val_mpa, val_miou, val_fwiou])
                
                if  i+1 == total_step and epoch%3==0 and val_miou>0.8 :
                    print("Saving the model at Epoch ",str(epoch+1)," with Val IOU ",str(val_miou))
                    torch.save(unet.state_dict(), 
                                './trained_models/unet_'+opt+'_'+los+'_'+str(epoch+1)+'.pkl')
                    print("Model saved : ")

            print("Epoch: ", epoch, " Loss: ",totalloss.item()/len(train_dataset), "Validation IOU: ", val_miou)
            history_np = np.array(history)
            np.save('./logs/unet_'+opt+'_'+los+'.npy',history_np)
        end = time.time()
        print(("Total time taken for training : ",(end-start)/60," Minutes"))

            
            
            
            

Using loss function:  bce  & Optimiser:  adam


KeyboardInterrupt: 

In [ ]:
#from torchvision import models
#print(unet) #i do not know if this works yet

In [11]:
torch.save(unet.state_dict(), './trained_models/unet_adam_bce_final.pkl') #save the final model, just as final check

In [12]:
#Prediction

import os
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from PIL import Image
import numpy as np




torch.cuda.manual_seed(777)
torch.manual_seed(777)

img_folder = 'isic2018_100_20_30_data/test/images/'
img_dir = os.listdir(img_folder)
img_list = [img_folder+k for k in img_dir]
img_list.sort()

unet = Unet(3,2).cuda()
unet.load_state_dict(torch.load('isic2018_100_20_30_trained_models/unet_'+opt+'_'+los+'_'+'final'+'.pkl'))


for file in img_list:
    img = Image.open(file).resize([512,512])
    img = transform(img).cuda().unsqueeze(0)
    with torch.no_grad():
        pred = unet(img)
    pred = torch.argmin(pred,1)
    pred = pred.squeeze().cpu().numpy()
    pred = np.uint8(pred*255)
    pred_img = Image.fromarray(pred)
    img_name = str.split(file, '/')[-1]
    
    img_name = 'isic2018_100_20_30_data/test/unet_pred/' + img_name
    pred_img.save(img_name, 'png')


FileNotFoundError: [WinError 3] The system cannot find the path specified: 'isic2018_100_20_30_data/test/images/'

In [ ]:
#Concattenate Test Image, Ground Truth, Predicted Mask,

for image in os.listdir("isic2018_100_20_30_data/test/images") :
  #print(image)
  original = PIL.Image.open("isic2018_100_20_30_data/test/images/"+image)
  #print(original)
  print(image.split(".png")[0]+"_segmentation.png")
  groundTruth = PIL.Image.open("isic2018_100_20_30_data/test/masks/"+image.split(".jpg")[0]+"_segmentation.png")
  unetPrediction = PIL.Image.open("isic2018_100_20_30_data/test/unet_pred/"+image)
  concat = PIL.Image.new('RGB',(original.width + groundTruth.width + unetPrediction.width, original.height))
  concat.paste(original,(0,0))
  concat.paste(groundTruth,(original.width,0))
  concat.paste(groundTruth,(original.width + groundTruth.width,0))
  concat.save("isic2018_100_20_30_data/test/results/"+image)



In [ ]:
concat #check one image